### Reading all VCF data and saving it as CSV files

This notebook is here to convert VCF files to something more familiar and easier to use - CSV files. However, I do not claim to have enough expertise in working with VCF files to say if this makes sense and if I have done it correctly here. As such, I would like to ask other team members to review this code, check if such transformation even makes sense and improve this notebook as they see fit.

In [1]:
!pip install "setuptools<58" --upgrade
!pip uninstall pyvcf
!pip install pyvcf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
arviz 0.15.1 requires setuptools>=60.0.0, but you have setuptools 57.5.0 which is incompatible.
cvxpy 1.3.2 requires setuptools>65.5.1, but you have setuptools 57.5.0 which is incompatible.


  Preparing metadata (setup.py) ... done
  Created wheel for pyvcf: filename=PyVCF-0.6.8-cp310-cp310-linux_x86_64.whl size=174265 sha256=6f62a567c25d33757750fddae1d38b530818cdd133b560904fdc38b1de4b4d72
  Stored in directory: /root/.cache/pip/wheels/da/88/3b/b691691894da44fc597c93ba7f953a46d1413ed7411906a6d2
Successfully built pyvcf


In [1]:
file1 = "/content/drive/MyDrive/Pathogenicity_Data/EE_015.vcf.gz"
file2 = "/content/drive/MyDrive/Pathogenicity_Data/EE_050.vcf.gz"
file3 = "/content/drive/MyDrive/Pathogenicity_Data/EE_069.vcf.gz"

In [3]:
import vcf
import pandas as pd

In [58]:
def convert_to_csv_test(vcf_file):
    data = []
    data2 = []

    with open(vcf_file, 'r') as vcf_file_binary:
        vcf_reader = vcf.Reader(vcf_file_binary)
        column_headers = ["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER"]

        column_headers.extend(list(vcf_reader.infos.keys())[:-1])
        column_csq_headers = vcf_reader.infos["CSQ"].desc[50:].split("|")

        for record in vcf_reader:
            row = []
            row.extend([record.CHROM, record.POS, record.ID, record.REF, record.ALT, record.QUAL, record.FILTER])

            for field in vcf_reader.infos.keys():
                if field != "CSQ":
                    info_value = record.INFO.get(field)
                    row.append(info_value)
                else:
                    data2.append(record.INFO.get(field)[0].split("|"))

            data.append(row)

    df1 = pd.DataFrame(data, columns=column_headers)
    df2 = pd.DataFrame(data2, columns=column_csq_headers)
    return df1, df2

In [59]:
df1, df2 = convert_to_csv_test("data/EE_sample/EE_sample.vcf")
for i, x in enumerate(df1.iloc[0]):
    print(f"KEY: {df1.columns[i]:25} VAL:{x}")

print("-----------------------------------------------------")
for i, x in enumerate(df2.iloc[0]):
    print(f"KEY: {df2.columns[i]:25} VAL:{x if x != '' else None}")

KEY: CHROM                     VAL:chr1
KEY: POS                       VAL:15820
KEY: ID                        VAL:rs2691315
KEY: REF                       VAL:G
KEY: ALT                       VAL:[T]
KEY: QUAL                      VAL:None
KEY: FILTER                    VAL:[]
KEY: ACMG_class                VAL:['Uncertain%40Significance']
KEY: ACMG_coding_impact        VAL:['non%40coding']
KEY: ACMG_gene                 VAL:None
KEY: ACMG_rules                VAL:['BP4']
KEY: ACMG_score                VAL:[0.193]
KEY: ACMG_transcript           VAL:None
KEY: AMP_matches               VAL:['V']
KEY: AMP_rules                 VAL:['Freq_II%2CType_IV%2CPred_IV']
KEY: AMP_score                 VAL:None
KEY: AMP_tier                  VAL:['Tier%40IV']
KEY: AMP_total_samples         VAL:[0]
KEY: AS_FilterStatus           VAL:SITE
KEY: AS_SB_TABLE               VAL:12%2C23|7%2C2
KEY: CGDinheritance            VAL:None
KEY: ClinVarClass              VAL:None
KEY: ClinVarDisease            VA

In [56]:
df1

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,ACMG_class,ACMG_coding_impact,ACMG_gene,...,gnomadExomes_AF_ethnic,gnomadExomes_AN,gnomadGenomesAC,gnomadGenomesAN,gnomadGenomesEthnic_AC_Hom,gnomadGenomes_AC_Hemi,gnomadGenomes_AC_Hom,gnomadGenomes_AF,gnomadGenomes_AF_ethnic,hgvs
0,chr1,15820,rs2691315,G,[T],None,[],[Uncertain%40Significance],[non%40coding],None,...,None,None,[18059],[82218],[233],None,[18059],[0.219648],[0.0938074],None
1,chr1,17385,rs201535981,G,[A],None,[],[Uncertain%40Significance],[non%40coding],None,...,[0.199722],[53936],[21384],[111582],[1],None,[21384],[0.191644],[0.213724],None
2,chr1,17697,rs71260069,G,[C],None,[],[Benign],[non%40coding],None,...,None,None,[17539],[116050],[2],None,[17539],[0.151133],[0.178196],None
3,chr1,133129,rs367730352,G,[A],None,[],[Uncertain%40Significance],[non%40coding],None,...,None,None,[72154],[131698],[7468],None,[72154],[0.547875],[0.597389],None
4,chr1,183629,rs71267774,G,[A],None,[],[Benign],[non%40coding],None,...,None,None,[14902],[111796],[1],None,[14902],[0.133296],[0.220891],None
5,chr1,184267,rs1396210256,C,[T],None,[],[Likely%40Benign],[non%40coding],None,...,None,None,[887],[144780],[0],None,[887],[0.00612654],[0.00631038],None


In [57]:
df2

,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,...,SpliceAI_pred_DP_DG,SpliceAI_pred_DP_DL,SpliceAI_pred_DS_AG,SpliceAI_pred_DS_AL,SpliceAI_pred_DS_DG,SpliceAI_pred_DS_DL,SpliceAI_pred_SYMBOL,LOEUF,PHENOTYPES,NMD
0,T,downstream_gene_variant,MODIFIER,DDX11L1,ENSG00000223972,Transcript,ENST00000450305.2,transcribed_unprocessed_pseudogene,,,...,,,,,,,,,,
1,A,downstream_gene_variant,MODIFIER,DDX11L1,ENSG00000223972,Transcript,ENST00000450305.2,transcribed_unprocessed_pseudogene,,,...,,,,,,,,,,
2,C,downstream_gene_variant,MODIFIER,DDX11L1,ENSG00000223972,Transcript,ENST00000450305.2,transcribed_unprocessed_pseudogene,,,...,,,,,,,,,,
3,A,non_coding_transcript_exon_variant,MODIFIER,CICP27,ENSG00000233750,Transcript,ENST00000442987.3,processed_pseudogene,1/1,,...,,,,,,,,,,
4,A,downstream_gene_variant,MODIFIER,MIR6859-2,ENSG00000273874,Transcript,ENST00000612080.1,miRNA,,,...,,,,,,,,,,
5,T,downstream_gene_variant,MODIFIER,MIR6859-2,ENSG00000273874,Transcript,ENST00000612080.1,miRNA,,,...,,,,,,,,,,


In [ ]:
df1.to_csv('data/EE_sample/EE_sample.csv', index=False)
df2.to_csv('data/EE_sample/EE_sample_csq.csv', index=False)

In [8]:
def convert_to_csv(vcf_file):
    data = []

    with open(vcf_file, 'rb') as vcf_file_binary:
        vcf_reader = vcf.Reader(vcf_file_binary)
        column_headers = ["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "FORMAT"]

        column_headers.extend(vcf_reader.infos.keys())
        column_headers.append("SAMPLE")

        for record in vcf_reader:
            row = []
            row.extend([record.CHROM, record.POS, record.ID, record.REF, record.ALT, record.QUAL, record.FILTER, record.FORMAT])

            for field in vcf_reader.infos.keys():
                info_value = record.INFO.get(field)
                row.append(info_value)

            for sample in record.samples:
                row.append(sample['GT'])

            data.append(row)

    df = pd.DataFrame(data, columns=column_headers)
    return df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,FORMAT,ACMG_class,ACMG_coding_impact,...,gnomadGenomesAC,gnomadGenomesAN,gnomadGenomesEthnic_AC_Hom,gnomadGenomes_AC_Hemi,gnomadGenomes_AC_Hom,gnomadGenomes_AF,gnomadGenomes_AF_ethnic,hgvs,CSQ,SAMPLE
0,chr1,15820,rs2691315,G,[T],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,[Uncertain%40Significance],[non%40coding],...,[18059],[82218],[233],None,[18059],[0.219648],[0.0938074],None,[T|downstream_gene_variant|MODIFIER|DDX11L1|EN...,0/1
1,chr1,17385,rs201535981,G,[A],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,[Uncertain%40Significance],[non%40coding],...,[21384],[111582],[1],None,[21384],[0.191644],[0.213724],None,[A|downstream_gene_variant|MODIFIER|DDX11L1|EN...,0/1
2,chr1,17697,rs71260069,G,[C],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,[Benign],[non%40coding],...,[17539],[116050],[2],None,[17539],[0.151133],[0.178196],None,[C|downstream_gene_variant|MODIFIER|DDX11L1|EN...,0/1
3,chr1,133129,rs367730352,G,[A],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,[Uncertain%40Significance],[non%40coding],...,[72154],[131698],[7468],None,[72154],[0.547875],[0.597389],None,[A|non_coding_transcript_exon_variant|MODIFIER...,0/1
4,chr1,183629,rs71267774,G,[A],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,[Benign],[non%40coding],...,[14902],[111796],[1],None,[14902],[0.133296],[0.220891],None,[A|downstream_gene_variant|MODIFIER|MIR6859-2|...,0/1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102512,chrY,56856512,rs76439603,A,[C],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:PGT:PID:PS:SB,[Uncertain%40Significance],[non%40coding],...,None,None,None,None,None,None,None,None,[C|downstream_gene_variant|MODIFIER|CTBP2P1|EN...,0/1
102513,chrY,56856517,rs75739861,A,[G],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:PGT:PID:PS:SB,[Uncertain%40Significance],[non%40coding],...,None,None,None,None,None,None,None,None,[G|downstream_gene_variant|MODIFIER|CTBP2P1|EN...,0/1
102514,chrY,56856554,rs77088808,T,[G],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,[Uncertain%40Significance],[non%40coding],...,None,None,None,None,None,None,None,None,[G|downstream_gene_variant|MODIFIER|CTBP2P1|EN...,0/1
102515,chrY,56856581,rs9320083,G,[A],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,[Uncertain%40Significance],[non%40coding],...,[2],[135],[0],None,[2],[0.0148148],[0.0],None,[A|downstream_gene_variant|MODIFIER|CTBP2P1|EN...,0/1


In [ ]:
EE_015=convert_to_csv(file1)
EE_015.to_csv('/content/drive/MyDrive/Pathogenicity_Data/EE_015.csv', index=False)

In [ ]:
EE_050=convert_to_csv(file2)
EE_050.to_csv('/content/drive/MyDrive/Pathogenicity_Data/EE_050.csv', index=False)

In [5]:
EE_069=convert_to_csv(file3)
EE_069.to_csv('/content/drive/MyDrive/Pathogenicity_Data/EE_069.csv', index=False)